# Kütüphanelerin İndirilmesi

In [ ]:
pip install osmnx

In [ ]:
pip install plotly_express

In [ ]:
pip install feather-format

# Import

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import feather
import osmnx
from shapely.geometry import Point, LineString
import plotly_express as px
import networkx as nx
import re
from shapely.geometry import Point
import numpy as np
osmnx.config(use_cache=True, log_console=True)

# Verilerin Okunması

## Trafik Verisi

In [ ]:
fuseed_data = pd.read_csv("Fuseed_Data_İlçe.csv")

In [ ]:
df_1 = pd.read_excel("Trafik Verisi-4 İlçe.xlsx", sheet_name="Beyoğlu")

Birleşik veride sadece Beyoğlu'na denk gelen yolları seçiyoruz.

In [ ]:
df_2 = fuseed_data[fuseed_data.vSegID.isin(df_1.vSegID)]

In [ ]:
df = df_1.append(df_2)

In [ ]:
list(df.columns)

Yolların segment idlerini okuyoruz.

In [ ]:
segments = pd.read_excel("Trafik Verisi-4 İlçe.xlsx", sheet_name="Beyoğlu SegmenID")

## Hava Durumu Verisi

In [ ]:
weather = pd.read_csv("Istanbul Weather Data.csv")

In [ ]:
weather['DateTime'] = pd.to_datetime(weather['DateTime'],format = "%d.%m.%Y")

Hız verilerinin olduğu tarih aralığındaki hava durumu bilgisini seçiyoruz.

In [ ]:
weather = weather[(weather['DateTime'] >= '2019-01-01') & (weather['DateTime'] <= '2019-06-30')] 

In [ ]:
list(weather.columns)

Günlük yağmur miktarlarını dictionary olarak çeviriyoruz.

In [ ]:
rain_dict = dict(zip(weather['DateTime'].dt.date,weather['Rain']))

## Tatil Tarihleri

Tatil tarihlerini okuyoruz.

In [ ]:
holidays = pd.read_excel("resmi_tatiller.xlsx")
holiday_dates = list(holidays['tarih'])

## Bakım Verileri

Bakım verilerini okuyoruz.

In [ ]:
maintenances_1 = pd.read_csv("yol_bakım.csv")
maintenances_1.head()

In [ ]:
maintenances_2 = pd.read_csv("yol_bakim_2.csv")
maintenances_2.head()

In [ ]:
maintenances = pd.concat([maintenances_1, maintenances_2], ignore_index=True)

Sadece Beyoğlu'nda olan bakımları alıp, bakımları sıralıyoruz.

In [ ]:
maintenances = maintenances[maintenances["ilce"] == 'BEYOĞLU']
maintenances.sort_values(by=['tarih','ilce'],inplace=True)
maintenances = maintenances.reset_index().drop('index',axis=1).copy()
maintenances

# Veri Düzenlemesi

## Trafik ve Segment Verisinin Düzenlemesi

### Segment Verisinin Düzenlenmesi

In [ ]:
def listAppend(x):
    """ If a list contains two strings, concatenates them and returns a single one. """
    if len(x) == 2 :
        return x[0] + x[1]
    return x[0]

In [ ]:
def meanCoord(x):
    """ Calculate mean of given coordinates for some road. """
    latSum = 0
    lonSum = 0
    length = len(x)
    for (lat,lon) in x:
        latSum += lat 
        lonSum += lon
    return(latSum/length , lonSum/length) # returns mean latitude and longitude

In [ ]:
def calculateCoordinates(segments = segments):
    """ Arranges coordinate strings and finds the mean coordinate. """
    segments = segments.groupby('vsegıd')['shape'].agg(','.join).reset_index() # concatenate shape information
    segments['shape'] = segments['shape'].apply(lambda x : x.split("LINESTRING ( "))
    segments['shape'] = segments['shape'].apply(lambda x :[elem.strip() for elem in x if elem])
    segments['shape'] = segments['shape'].apply(lambda x :[elem.split(", ") for elem in x ])
    segments['shape'] = segments['shape'].apply(lambda x :listAppend(x) )
    segments['shape'] = segments['shape'].apply(lambda x : [elem.replace(',', '' ) for elem in x])
    segments['shape'] = segments['shape'].apply(lambda x : [(float(elem.split()[1]), float(elem.split()[0])) for elem in x] )
    segments['shape'] = segments['shape'].apply(lambda x : meanCoord(x))
    return segments

Her segment için ortalama bir koordinat çıkarıyoruz.

In [ ]:
coords = calculateCoordinates(segments)

In [ ]:
coords.head()

### Trafik Verisinin Düzenlenmesi

"0-30 DK"yi 15'inci , "30-60 DK"yi 45'inci dakika olarak işaretliyoruz.

In [ ]:
GRPdict = {"0-30DK": 15 , "30-60DK": 45}
df['GRP'] = df['GRP'].apply(lambda x : GRPdict[x])

Sütun adlarını değiştirdikten sonra datetime sütunu oluşturuyoruz.

In [ ]:
df.rename(columns={'fusedYear': 'year', 'fusedMonth': 'month', 'fusedday' : 'day', 'fusedHour' : 'hour' , 'GRP' : 'minute'}, inplace=True)
df['datetime'] = pd.to_datetime(df[['year','month','day','hour','minute']])

Ayları sayıdan ada çeviriyoruz.

In [ ]:
df['month'] = pd.to_datetime(df['month'], format='%m').dt.month_name().str.slice(stop=3)

Weekday ve resmi tatil parametrelerini oluşturuyoruz.

In [ ]:
df['weekday'] = df['datetime'].dt.day_name()
df['holiday'] = df['datetime'].apply(lambda x : 1 if str(x.date()) in holiday_dates else 0)

Resmi tatillere bakıyoruz.

In [ ]:
df[df['holiday']==1].head()

Covid öncesi/sonrası bilgisini işaretliyoruz.

In [ ]:
covid_start = pd.to_datetime('2019-03-16')
df['covid'] = df['datetime'].apply(lambda date: 1 if date >= covid_start else 0)

Trafik verisine gece/gündüz bilgisini ekliyoruz.

In [ ]:
df['gece'] = df['hour'].apply(lambda hour : 'Gündüz' if hour<=23 and hour>=6 else 'Gece')

O günkü yağış miktarını trafik verisine ekliyoruz.

In [ ]:
df['rain'] = df['datetime'].apply(lambda x: rain_dict[x.date()])

In [ ]:
df = df.reset_index().drop('index', axis=1) 

## Bakım Verisinin Düzenlenmesi

Bakımların ay, gün, gece/gündüz bilgisini hesaplıyoruz.

In [ ]:
maintenances['datetime'] = pd.to_datetime(maintenances['tarih'])
maintenances['month'] = maintenances['datetime'].dt.month_name().str.slice(stop=3)
maintenances['day'] = maintenances['datetime'].dt.day
maintenances['gece'] = maintenances['gece'].apply(lambda x : 'Gece' if x == "EVET" else 'Gündüz')

İhtiyaç olmayan bilgileri bakımlardan siliyoruz.

In [ ]:
maintenances.drop(['tarih','lat','lon','id','ilce','yol_adi'],axis = 1,inplace=True)
maintenances.head()

# Veri İncelemesi

## Hızların İncelenmesi

In [ ]:
plt.style.use('ggplot')

Yönlere göre hızları inceliyoruz.

In [ ]:
group_dir = df.groupby('vSegDir' ,axis = 0)
group_dir['avgspeed'].describe()

Segment idlere göre hızları inceliyoruz.

In [ ]:
group_id = df.groupby('vSegID' ,axis = 0)
group_id['avgspeed'].describe()

Ortalama hızlar için histogram çizdiriyoruz.

In [ ]:
plt.hist(df['avgspeed'], bins=50)
plt.xlabel("Average speed")
plt.ylabel("Frequency")
plt.title("Frequencies Of Speed Values")
plt.show()

Ortalama hızlar için boxplot çizdiriyoruz.

In [ ]:
plt.boxplot(df['avgspeed'])
plt.title('Boxplot of speed values')
plt.ylabel('Speed')
plt.show()

Saatlik ortalama hızları çizdiriyoruz.

In [ ]:
hour_means = df.groupby(df['datetime'].dt.hour).mean()
hour_means['indices'] = hour_means.index
plt.bar(hour_means['indices'],hour_means['avgspeed'])
plt.xlabel("Hour")
plt.ylabel("Average speed")
plt.title("Average Speed by Hour")
plt.show()

Haftanın günlerine göre ortalama hızları çizdiriyoruz.

In [ ]:
days = ('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday')
day_means = df.groupby(df['datetime'].dt.day_name()).mean().reindex(days[::-1])
day_means['indices'] = day_means.index
plt.barh(day_means['indices'],day_means['avgspeed'])
plt.xlabel("Average speed")
plt.ylabel("Weekday")
plt.title("Average Speed by Weekdays")
plt.show()

Aylara göre ortalama hızları çizdiriyoruz.

In [ ]:
months = ("January","February","March","April","May","June","July","August","September","October","November","December")
month_means = df.groupby(df['datetime'].dt.month_name()).mean().reindex(months[::-1])
month_means['indices'] = month_means.index
plt.barh(month_means['indices'],month_means['avgspeed'])
plt.xlabel("Average speed")
plt.ylabel("Month")
plt.show()

## Bakımların İncelenmesi

In [ ]:
maintenances['isin_cinsi'].value_counts().plot(kind = 'pie',ylabel = "") 

## Hava Durumlarının İncelenmesi

In [ ]:
weather['Condition'].value_counts().plot(kind = "barh")

In [ ]:
weather.head()

# Haritanın Oluşturulması

Beyoğlu için harita indiriyoruz.

In [ ]:
G = osmnx.graph_from_address("Beyoğlu, İstanbul", 5000, network_type= "drive") # download map for Beyoğlu

In [ ]:
remain_edges = set() # set of remaining edges.
remain_nodes = set() # set of remaining nodes.
segment_dict = {} # dictionary that stores corresponding edge for each segment id

In [ ]:
def match_edge(coordinate, seg_id):
    edge = osmnx.distance.get_nearest_edge(G,coordinate) # find edge nearest to the coordinate
    remain_edges.add(edge) 
    segment_dict[seg_id] = str(edge) 

In [ ]:
def graphSubset(G, remain_edges):
    """ Removes unmatched edges and nodes from the graph"""
    all_edges = set() # set of all edges in downloaded graph
    for edge in G.edges:
        all_edges.add(edge)

    all_nodes = set() # set of all nodes in downloaded graph
    for node in G.nodes:
        all_nodes.add(node)
    
    for edge in remain_edges:
        # nodes that are related to remaining edges must remain
        remain_nodes.add(edge[0]) 
        remain_nodes.add(edge[1])
        
    delete_nodes = all_nodes - remain_nodes # nodes to be deleted
    delete_edges = all_edges - remain_edges # edges to be deleted
    G.remove_edges_from(list(delete_edges)) # delete unnecessary edges
    G.remove_nodes_from(list(delete_nodes)) # delete unnecessary nodes

Her segmenti haritadaki bir edge ile eşleştiriyoruz.

In [ ]:
coords.apply(lambda coord: match_edge(coord['shape'],coord['vsegıd']), axis = 1) 
df['edge'] = df['vSegID'].apply(lambda seg : segment_dict[seg])

Eşleşmeyen yolları haritadan siliyoruz.

In [ ]:
graphSubset(G, remain_edges)

In [ ]:
def coord_extract(url):
    """ Extracts latitude and longitude from each maintenance url. """
    lat = float(re.search(r'(?<=lat=)[\d.]*' ,url)[0])
    lon = float(re.search(r'(?<=lon=)[\d.]*' ,url)[0])
    return (lat,lon)

Her bir bakımın koordinat bilgisi hesaplandıktan sonra, haritadaki en yakın yolla eşleştiriliyor.

In [ ]:
maintenances['coord'] = maintenances['harita'].apply(lambda url : coord_extract(url))
maintenances['edges'] = maintenances['coord'].apply(lambda x : osmnx.distance.get_nearest_edge(G,x)) # her koordinata karşılık gelen edgeleri bir listede topla. 
maintenances.drop('harita', axis=1, inplace=True)

In [ ]:
maintenance_edges = set()
maintenance_edges.update(maintenances['edges'])

# Feature Geliştirme

In [ ]:
df.drop('vSegID', inplace = True, axis = 1)
df.drop('vSegDir', inplace = True, axis = 1)
df.drop('minute', inplace = True, axis = 1)
df.drop('year', inplace = True, axis = 1)

Haritadaki edgeleri stringe çeviriyoruz.

In [ ]:
maintenances['edges'] = maintenances['edges'].apply(lambda x : str(x))

Gün, ay, yol ve gece-gündüz bilgisini birleştirerek bakımı tanımlayan bir parametre oluşturuyoruz.

In [ ]:
cols = ['day','month','edges','gece']
maintenances['combined'] = maintenances[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis = 1)

In [ ]:
maintenances.head()

Trafik verileri için de bakımlarınkine benzeyen tanımlayıcı bir parametre oluşturuyoruz.

In [ ]:
cols = ['day','month','edge','gece']
df['combined'] = df[cols].apply(lambda row: '_'.join((str(x) for x in row)),axis = 1)

In [ ]:
df.head()

In [ ]:
maintenance_dict = dict(zip(maintenances.combined, maintenances.isin_cinsi)) 

In [ ]:
df['maintenance'] = df['combined'].apply(lambda x : maintenance_dict[x] if x in maintenance_dict else "no_maintenance")

In [ ]:
ave = df.groupby(['month','day','hour','edge'])['avgspeed'].mean().reset_index()
ave

# Harita Görselleştirme

## Bakımlar

Bakım olan yolları işaretliyoruz.

In [ ]:
def draw_maintenances(maintenance_edges, Graph=G):
    """ Paints roads with maintenances as red. """
    color = ['r' if edge in maintenance_edges else 'w' for edge in G.edges(keys=True)]
    fig, ax = osmnx.plot_graph(G,edge_color= color)
    
draw_maintenances(maintenance_edges)

## Hız

Düşük ve yüksek hız olan yolları işaretliyoruz.

In [ ]:
def speed_tocolor(speed, threshold = 30):
    """ Returns red for low speed values and green for the others. """
    if speed > threshold:
        return 'g'
    else:
        return 'r'

In [ ]:
def color_map(month, day, hour, graph=G):
    """ Creates a color map for a given time. """
    hour_speed = df.groupby(['month','day','hour']) 
    draw_group = hour_speed.get_group((month,day,hour)) # find values at a specific time.
    speed_dict = dict(zip(draw_group.edge,draw_group.avgspeed)) # create a dictionary for speed values
    color = [speed_tocolor(speed_dict[str(edge)]) if str(edge) in speed_dict else 'g' for edge in G.edges]
    fig, ax = osmnx.plot_graph(G,edge_color= color)

Örnek olarak 19 Mart saat 18'deki hızları çizdiriyoruz.

In [ ]:
color_map('Mar',19,18)

# Model İçin Hazırlık

Modele eğitilebilecek haldeki dataframe için df_model değişkenini oluşturuyoruz.

In [ ]:
df_model = df.copy()

Modele girmesi gerekmeyen parametreleri siliyoruz.

In [ ]:
df_model = df_model.drop(['day','datetime','combined','gece'], axis=1)

0 olarak ölçülen ve 80'den yüksek çıkan hızları çıkarıyoruz.

In [ ]:
df_model = df_model[(df_model.avgspeed > 0) & (df_model.avgspeed < 80)].reset_index().drop('index',axis = 1)

In [ ]:
df_model = pd.concat([df_model,pd.get_dummies(df_model['hour'])],axis=1).drop(['hour'],axis=1)
df_model = pd.concat([df_model,pd.get_dummies(df_model['weekday'])],axis=1).drop(['weekday'],axis=1)
df_model = pd.concat([df_model,pd.get_dummies(df_model['edge'])],axis=1).drop(['edge'],axis=1)
df_model = pd.concat([df_model,pd.get_dummies(df_model['maintenance'])],axis=1).drop(['maintenance'],axis=1)
df_model = pd.concat([df_model,pd.get_dummies(df_model['month'])],axis=1).drop(['month'],axis=1)

In [ ]:
df_model.head()

In [ ]:
df_model.drop("no_maintenance", axis = 1, inplace = True)

In [ ]:
df_model.columns = df_model.columns.astype(str)
df_model.to_feather("beyoğluReady.feather") 

In [ ]:
y = df_model['avgspeed'].values
X = df_model.drop(['avgspeed'], axis=1)

In [ ]:
X.shape

# Modellerin Eğitilmesi

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=45)

## Decision Tree Regressor

In [ ]:
dtRegressor = DecisionTreeRegressor(max_depth= 110, min_samples_leaf = 25) 

In [ ]:
dtRegressor.fit(X_train,y_train)

In [ ]:
y_pred_dt = dtRegressor.predict(X_test)

In [ ]:
mean_absolute_error(y_test,y_pred_dt)

In [ ]:
r2_score(y_test,y_pred_dt)

Residual plotu belli bir aralık için çizdiriyoruz.

In [ ]:
residuals_dt = y_test - y_pred_dt
plt.scatter(y_pred_dt[9000:9900],residuals_dt[9000:9900])
plt.hlines(y=0,xmin = 0, xmax=80, color ="red", linestyles = "dashed")
plt.ylabel("Residual")
plt.xlabel("Prediction")
plt.title("Residual Plot")
plt.show()

Feature importance'ı 0 olan parametreleri inceliyoruz.

In [ ]:
zeros = np.where(dtRegressor.feature_importances_== 0)
X.columns.values[zeros]

## Random Forest Regressor

In [ ]:
rfRegressor = RandomForestRegressor(max_depth=60, min_samples_leaf = 30, n_estimators = 10, max_samples = 0.5, bootstrap = True)

In [ ]:
rfRegressor.fit(X_train,y_train)

In [ ]:
y_pred_rf = rfRegressor.predict(X_test)

In [ ]:
mean_absolute_error(y_test,y_pred_rf)

In [ ]:
r2_score(y_test,y_pred_rf)

Residual plotu belli bir aralık için çizdiriyoruz.

In [ ]:
residuals_rf = y_test - y_pred_rf
plt.scatter(y_pred_rf[10000:10900], residuals_rf[10000:10900])
plt.hlines(y=0,xmin = 0, xmax=80, color ="red", linestyles = "dashed")
plt.ylabel("Residual")
plt.xlabel("Prediction")
plt.title("Residual Plot")
plt.show()

## Linear Regression

In [ ]:
lRegressor = LinearRegression()

In [ ]:
lRegressor.fit(X_train,y_train)

In [ ]:
y_pred_l = lRegressor.predict(X_test)

In [ ]:
mean_absolute_error(y_test,y_pred_l)

In [ ]:
r2_score(y_test,y_pred_l)

Residual plotu belli bir aralık için çizdiriyoruz.

In [ ]:
residuals_l = y_test - y_pred_l
plt.scatter(y_pred_l[9000:9900], residuals_l[9000:9900])
plt.hlines(y=0,xmin = 0, xmax=80, color ="red", linestyles = "dashed")
plt.ylabel("Residual")
plt.xlabel("Prediction")
plt.title("Residual Plot")
plt.show()

## Ridge Regression

Alpha = 0.5 değeriyle Ridge Regression modeli eğitiyoruz.

In [ ]:
ridgeModel = Ridge(alpha = 0.5) 

In [ ]:
ridgeModel.fit(X_train, y_train) 

In [ ]:
y_pred_r = ridgeModel.predict(X_test)

In [ ]:
mean_absolute_error(y_test,y_pred_r)

In [ ]:
r2_score(y_test,y_pred_r)

Residual plotu belli bir aralık için çizdiriyoruz.

In [ ]:
residuals_r = y_test - y_pred_r
plt.scatter(y_pred_r[9000:9900],residuals_r[9000:9900])
plt.hlines(y=0,xmin = 0, xmax=80, color ="red", linestyles = "dashed")
plt.ylabel("Residual")
plt.xlabel("Prediction")
plt.title("Residual Plot")
plt.show()

# Trafik Tahmini

Belli bilgiler girildiğinde trafiğin tahmin edilmesi için bir fonksiyon.

In [ ]:
def predictEffect(date, hour, vSegID, maintenance, rain = 0.0):
    # check holiday
    if date in holiday_dates:
        holiday = 1
    else:
        holiday = 0

    date = pd.to_datetime(date) 
    
    # mark covid data
    if date >= pd.to_datetime('2019-03-16'):
        covid = 1
    else:
        covid = 0
        
    weekday = date.day_name() # find corresponding weekday
    edge = str(segment_dict[vSegID]) # find edge name corresponding to segment id
    month = date.month_name()[:3] # month number to month name abbreviation

    no_maintenance = pd.DataFrame(0, index=[0], columns= list(X.columns)) # create a row filled with 0's
    
    # insert the data of the query.
    no_maintenance['resmi_tatil'] = no_maintenance['resmi_tatil'].replace([0],resmi_tatil)
    no_maintenance['covid'] = no_maintenance['covid'].replace([0],covid)
    no_maintenance[hour] = no_maintenance[hour].replace([0],1)
    no_maintenance[weekday] = no_maintenance[weekday].replace([0],1)
    no_maintenance[edge] = no_maintenance[edge].replace([0],1)
    no_maintenance[month] = no_maintenance[month].replace([0],1)
    no_maintenance['rain'] = no_maintenance['rain'].replace([0],rain)
    
    with_maintenance= no_maintenance.copy()  # copy the no maintenance version.
    with_maintenance[maintenance] = with_maintenance[maintenance].replace([0],1) # add maintenance.
    
    # predict with decision tree regressor defaultly.
    no_maintenance_prediction = dtRegressor.predict(no_maintenance)
    with_maintenance_prediction = dtRegressor.predict(with_maintenance)
    
    # if decision tree regressor does not make a distinction for maintenance, predict with ridge regressor
    if(with_maintenance_prediction == no_maintenance_prediction):
        no_maintenance_prediction = ridgeModel.predict(no_maintenance)
        with_maintenance_prediction = ridgeModel.predict(with_maintenance)
    print("Predicted average speed when there is no maintenance: ", no_maintenance_prediction)
    print("Predicted average speed when there is maintenance: ", with_maintenance_prediction)

In [ ]:
# predictEffect('2019-02-16', '6', vSegID=548, maintenance="UTK", rain=10) # an example prediction.